In [3]:
#importing required packages for data cleaning and for posgres connections

import pandas as pd
import psycopg2
from datetime import datetime

# To load data in pandas

data = pd.read_csv(r"D:\guvi\Projects\traffic_stops - traffic_stops_with_vehicle_number.csv") #r keyword is used to make raw string


C:\Users\sakth\AppData\Local\Temp\ipykernel_8992\872405347.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"D:\guvi\Projects\traffic_stops - traffic_stops_with_vehicle_number.csv") #r keyword is used to make raw string


In [4]:
# Dropping all columns with missing values values

data.dropna(axis=1, how="all", inplace=True)

#Filling NaN with suitable values

data.fillna({
    'driver_age': data['driver_age'].median,
    'country_name': 'unknown',
    'violation': 'unknown',
    'search_type': 'none',
    'stop_outcome': 'unknown',
    'stop_duration': 'unknown',
    'vehicle_number': 'unknown'
}, inplace=True)

# data["country_name"].fillna("unknown", inplace=True)
# data["driver_age"].fillna(data["driver_age"].median, inplace=True)
# data["violation"].fillna("unknown", inplace=True)
# data["search_type"].fillna("none", inplace=True)
# data["stop_outcome"].fillna("unknown", inplace=True)
# data["stop_duration"].fillna("unknown", inplace=True)
# data["vehicle_number"].fillna("unknown", inplace=True)

# converting the row into corrected date format 

data["stop_date"] = pd.to_datetime(data["stop_date"], errors = 'coerce').dt.strftime("%Y-%m-%d")

# converting the column into corrected time format

data["stop_time"] = pd.to_datetime(data["stop_time"],errors="coerce").dt.strftime("%H:%M:%S")

# converting date and time to timestamp

data['timestamp'] = pd.to_datetime(data['stop_date']+' '+data['stop_time'])



C:\Users\sakth\AppData\Local\Temp\ipykernel_8992\2406573683.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["stop_time"] = pd.to_datetime(data["stop_time"],errors="coerce").dt.strftime("%H:%M:%S")


In [5]:
data

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number,timestamp
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473,2020-01-01 00:00:00
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441,2020-01-01 00:01:00
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264,2020-01-01 00:02:00
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807,2020-01-01 00:03:00
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305,2020-01-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568,2020-02-15 12:13:00
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098,2020-02-15 12:14:00
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328,2020-02-15 12:15:00
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,none,Warning,True,0-15 Min,True,KA24UZ8488,2020-02-15 12:16:00


In [9]:
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT    # for auto comitting postgres sql 

connection = psycopg2.connect(             #connection data
    host = "localhost",
    user = "postgres",
    password = "934446",
    port = 5432
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)      # for auto comitting postgres sql 

mediater = connection.cursor()   #defining the cursor with an variable name

mediater.execute("create database policepostregister")    # execute statement is used to run the query

In [10]:
from sqlalchemy import create_engine     # sqlalchemy is used to push the data to sql

host = "localhost"
user = "postgres"
password = "934446"
port = 5432
database = "policepostregister"

engine_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"   # f string must be used while loading data into postgres sql

engine = create_engine(engine_string)

data.to_sql("Policelog",engine,if_exists="replace",index=False)   # Defining a table name and telling to replace if the table name is already available

print("success")


success
